In [35]:
import pandas as pd
import numpy as np
import re
import requests
import seaborn as sns

In [36]:
hotels = pd.read_csv('./data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [37]:
#Заполняем пропуски
hotels = hotels.fillna(0)

In [38]:
#Удаляем дубликаты 
hotels = hotels.drop_duplicates()

---

Преобразование тегов

In [39]:
# Пркобразование тегов к списку
def tag_reconstr(data):
    temp_list = []
    for i in data.split("'"):
        temp_list.append(i.strip())

    temp_list.remove('[')
    temp_list.remove(']')

    for i in temp_list:
        if i == ',':
            temp_list.remove(i)
    return temp_list


tags_series = hotels['tags'].apply(tag_reconstr)
hotels['tags'] = hotels['tags'].apply(tag_reconstr)

#Формируем базу для one_hot, параллельно выдергиваем фит прожитых ночей
all_tags = tags_series.explode()
stayed_night_mask = all_tags.str.contains('Stayed', regex=False)
stayed_night = all_tags[stayed_night_mask]
stayed_night.str.split(' ')

def night_at_hotel(data):
    temp_var = 0
    for i in data.split(' '):
        try:
            temp_var = int(i)
        except ValueError:
            pass
    return temp_var

nights = stayed_night.apply(night_at_hotel)
nights = nights.rename('nights')
hotels = pd.concat([hotels, nights], axis=1)
hotels['nights'] = hotels['nights'].fillna(hotels['nights'].mode()[0])

#Вытаскиваем топ 50 тегов и разворачиваем на их основе one_hot
top_tags = all_tags.value_counts()[:51]
mask = top_tags.index.str.contains('Stayed', regex=False)
one_hot_columns = list(top_tags[~mask].index)

for element in one_hot_columns:
    hotels[element] = hotels['tags'].apply(lambda x: 1 if element in x else 0)

Кодируем отели

In [40]:
#Кодируем названия отелей
hotels['hotel_name'] = hotels['hotel_name'].astype('category')
hotels['hotel_name'] = hotels['hotel_name'].cat.codes

Код отеля по коду его страны

In [41]:
# Достаем код страны
response = requests.get('https://countrycode.org/')
country_code = pd.read_html(response.text)
country_code = country_code[0]
country_code = country_code[['COUNTRY', 'COUNTRY CODE']]

# Создаем функцию для формирования кодовой строки исходя из принципа территориальной близости принадлежности для кодов формата Х-ХХХ
def get_code(data):
    data = data.split('-')
    return int(data[0])


country_code['COUNTRY CODE'] = country_code['COUNTRY CODE'].apply(get_code)

# Формируем новый закодированный (в последствие) признак местоположения отеля
hotels['hotel_country'] = hotels['hotel_address'].str.split(' ')
hotels['hotel_country'] = hotels['hotel_country'].apply(lambda x: x[-2::])
hotels['hotel_country'] = hotels['hotel_country'].apply(
    lambda x: x[0]+' '+x[1] if x[0] == 'United' else x[1])

hotels = hotels.merge(
    country_code, how='left',
    left_on='hotel_country', right_on='COUNTRY'
)
hotels = hotels.rename(columns={'COUNTRY CODE': 'hotel_country_code'})


Вытаскиваем день недели и отдельно месяц

In [42]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['month'] = hotels['review_date'].dt.month
hotels['day_of_week'] = hotels['review_date'].dt.day_of_week

Проверяем на мультиколлинеарность

In [43]:
whole_corr = hotels.corr()[(hotels.corr() > 0.7) | (hotels.corr() < -0.7)]
whole_corr

/var/folders/1_/hgb3xg852md0hqqs3ybwbsp40000gn/T/ipykernel_22355/991658011.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  whole_corr = hotels.corr()[(hotels.corr() > 0.7) | (hotels.corr() < -0.7)]


,additional_number_of_scoring,average_score,hotel_name,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,lat,lng,...,Standard Queen Room,Deluxe Single Room,Small Double Room,Queen Guest Room,Deluxe Twin Room,King Room,Basic Double Room,hotel_country_code,month,day_of_week
additional_number_of_scoring,1.000000,NaN,NaN,NaN,0.824494,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
average_score,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hotel_name,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_total_negative_word_counts,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_number_of_reviews,0.824494,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_total_positive_word_counts,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_number_of_reviews_reviewer_has_given,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviewer_score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#Список фитов с мультиколлинеарностью
direct_corr = whole_corr.count(axis=0)
corr_columns = pd.DataFrame({'Name':direct_corr.index, 'CNT':direct_corr.values})
corr_columns.query("CNT > 1")

,Name,CNT
0,additional_number_of_scoring,2
4,total_number_of_reviews,2
11,Leisure trip,2
15,Business trip,2


In [45]:
hotels.drop(['additional_number_of_scoring', 'Business trip'], axis=1, inplace=True)
#Дропаем столбец даты что бы модель могла работать
hotels.drop(['review_date'], axis=1, inplace=True)

---

In [46]:
#Дропаем строковые фиты
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

In [47]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [48]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [49]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)


In [51]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.13626282894326716


MAPE: 0.13733658225339634